In [53]:
import numpy as np
import pandas as pd

import bct
import glob
import os
import networkx as nx

from pathlib import Path


In [129]:
def module_nodes(modules, module):
    nodes = []
    
    for (key, value) in modules.items():
        if value == module:
            nodes.append(key)
            
    return nodes

In [94]:
def relative_degree(grid, nodes, node):
    all_nodes = nodes
    if not np.isin(node, nodes):
        all_nodes.append(node)
    
    return (grid.subgraph(nodes).degree(node))

In [56]:
def global_degree(node, grid):
    return(grid.degree(node))

In [112]:
def participation_coefficient(grid, modules, node):
    acum = 0
    number_of_modules = len(set(list(modules.values())))
    
    for module in range(1,number_of_modules):
        relative_degree_value = relative_degree(grid, module_nodes(modules, module), node)
        global_degree_value = global_degree(node, grid)        
        
        if global_degree_value > 0:
            acum = pow(relative_degree_value / global_degree_value, 2)
        else:
            #print("Global degree zero for node %s and module %s", node, module)
            acum = 0
            
    return (1-acum)        

In [135]:
def intra_modular_degree_zscore(grid, modules, node):
    module = modules[node]    
    nodes = module_nodes(modules, module)
    relative_degrees = {}
    
    for  node in nodes:
        relative_degrees[node] = relative_degree(grid, nodes, node)
    
    im_degree = relative_degrees[node]
    degrees_values = list(relative_degrees.values())
    im_mean = np.mean(degrees_values)        
    im_std_deviation = np.std(degrees_values)
    
    if im_std_deviation > 0:
        return ((im_degree - im_mean) / im_std_deviation)
    else:
        return (0)
    

In [59]:
def node_clasfication(P_c, Z_c, P_i, Z_i):
    if (P_i > P_c and Z_i > Z_c):
        return("hub")
    if (P_i < P_c and Z_i > Z_c):
        return("provincial_hub")
    if (P_i < P_c and Z_i < Z_c):
        return("provincial_node")
    if (P_i > P_c and Z_i < Z_c):
        return("connector_node")
    

In [133]:
def node_roles(G, P_c, Z_c):
    result = {}
    modules = community_louvain.best_partition(G)

    for node in G.nodes():
        Z_i = intra_modular_degree_zscore(G, modules, node)
        P_i = participation_coefficient(G, modules, node)
        role = node_clasfication(P_c, Z_c, P_i, Z_i)        
        result[node] = list([Z_i, P_i, role])
    
    return(result)

In [114]:
def subjectmatrix_to_graphs(filename, densities):
    """
    Toma un archivo csv con la matriz de input y devuelve una lista de grafos con la densidad de aristas correspondiente
    """

    correlation_matrix = pd.read_csv(
        filename,
        header=None
    )
    n = correlation_matrix.shape[0]
    correlation_matrix -= np.diag(np.ones(n))
    max_links = (n*n-n)//2
    tril_idx = np.tril_indices(n,-1)
    sorted_correlations = sorted(
        correlation_matrix.values[tril_idx].flatten(),
        reverse=True
    )

    Gs = []
    for d in densities:
        idx = int(d*max_links)
        threshold = sorted_correlations[idx]
        Gs.append(nx.from_pandas_adjacency(correlation_matrix>=threshold))

    return Gs

In [62]:
def extract_graphs(tipo_red, densities):

    path = r'../resources/DataSujetos' # use your path
    all_files = glob.glob(path + "/{}_suj*.csv".format(tipo_red))


    graphs = {}
    for filename in all_files:
        Gs = subjectmatrix_to_graphs(filename, densities)
        name = Path(filename).stem
        graphs[name] = Gs

    return graphs

In [63]:
densities = np.linspace(0.0,0.4,10)

G_N1 = extract_graphs("N1", densities)
G_N2 = extract_graphs("N2", densities)
G_N3 = extract_graphs("N3", densities)
G_W = extract_graphs("W", densities)

In [136]:
node_roles(G_N1["N1_suj1"][2], 0.05, 1)

0 -1.8261456014624704 1.0 connector_node
1 -1.4968597790160594 1.0 connector_node
2 -1.8261456014624704 1.0 connector_node
3 -1.8261456014624704 1.0 connector_node
4 -1.8261456014624704 1.0 connector_node
5 -1.8261456014624704 1.0 connector_node
6 -1.8261456014624704 1.0 connector_node
7 -1.8261456014624704 1.0 connector_node
8 -1.8261456014624704 1.0 connector_node
9 -1.8261456014624704 1.0 connector_node
10 -1.8261456014624704 1.0 connector_node
11 -1.8261456014624704 1.0 connector_node
12 -1.8261456014624704 1.0 connector_node
13 -1.8261456014624704 1.0 connector_node
14 -1.8261456014624704 1.0 connector_node
15 -1.8261456014624704 1.0 connector_node
16 -1.4968597790160594 1.0 connector_node
17 -1.4968597790160594 1.0 connector_node
18 -1.4968597790160594 1.0 connector_node
19 -1.4968597790160594 1.0 connector_node
20 -1.4968597790160594 1.0 connector_node
21 -1.4968597790160594 1.0 connector_node
22 -1.8261456014624704 1.0 connector_node
23 -1.8261456014624704 1.0 connector_node
24

{0: [-1.8261456014624704, 1.0, 'connector_node'],
 1: [-1.4968597790160594, 1.0, 'connector_node'],
 2: [-1.8261456014624704, 1.0, 'connector_node'],
 3: [-1.8261456014624704, 1.0, 'connector_node'],
 4: [-1.8261456014624704, 1.0, 'connector_node'],
 5: [-1.8261456014624704, 1.0, 'connector_node'],
 6: [-1.8261456014624704, 1.0, 'connector_node'],
 7: [-1.8261456014624704, 1.0, 'connector_node'],
 8: [-1.8261456014624704, 1.0, 'connector_node'],
 9: [-1.8261456014624704, 1.0, 'connector_node'],
 10: [-1.8261456014624704, 1.0, 'connector_node'],
 11: [-1.8261456014624704, 1.0, 'connector_node'],
 12: [-1.8261456014624704, 1.0, 'connector_node'],
 13: [-1.8261456014624704, 1.0, 'connector_node'],
 14: [-1.8261456014624704, 1.0, 'connector_node'],
 15: [-1.8261456014624704, 1.0, 'connector_node'],
 16: [-1.4968597790160594, 1.0, 'connector_node'],
 17: [-1.4968597790160594, 1.0, 'connector_node'],
 18: [-1.4968597790160594, 1.0, 'connector_node'],
 19: [-1.4968597790160594, 1.0, 'connecto